# Types of Databases

<img src="https://bytesforlunch.files.wordpress.com/2011/01/relational-vs-non-relational.jpg?w=700">

### Relational
- Represent and store data in tables and rows.
- Use Structured Query Language (`SQL`), which is like a programming language for relational databases.
- Examples include `SQLite`, `MySQL`, `PostreSQL`, `T-SQL`.

### Non-relational
- Represent and store data as collections of documents, objects, key-value stores, or heirarchal data formats.
- Also known as "NoSQL" databases.
- Examples include `JSON`, `MongoDB`.

# SQL: Structured Query Language

- Programming language for storing, manipulating, and retreiving data stored in a relational database.
- All relational database management systems (e.g. `MySQL`, `SQLite`, `PostgreSQL`) use `SQL` as their standard database landuage.
- `SQL` can have different *dialects* that contain small, subtle differences (just like there are different *accents* in the english language). For example, a string-like data type in `MySQL` is defined as **`STRING`**, while in `SQLite` it is defined as **`TEXT`**. 
- However, the vast majority of `SQL` is the same in all relational database management systems.

# What will we learn?

In this lesson, we will learn how to retreive astronomical data from a `SQLite` database.

- How to retrieve data from a database
- How to select a subset of the data
- How to sort a query's results
- How to combine data from multiple tables

We will first learn how to do this through the `SQLite` command line interface, then we will learn how to accomplish the same tasks using Python.

# The `hipparcos.db` Database

In this lesson, we will use with the `hipparcos.db` database, which stores data from the Extended Hipparcos Compilation (XHIP).  

It contains:
- Spectral classificaitons
- Radial velocities
- Iron abundances
- Stellar classifications
- Stellar ages
- 2MASS photometry

The database can be downloaded from this URL:

https://www.dropbox.com/s/o80up2k4h3z7uud/hipparcos.db?dl=0

More information about the Extended Hipparcos Compilation:

- https://arxiv.org/abs/1108.4971
- http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=V%2F137D&target=readme&#sRM-F.42

This database contains two tables, named `data` and `photometry`.  The `data` table stores Astrometry, spectrography, space motions, and exoplanet indications.  The `photometry` tables stores photometry information.

# Opening and Navigating the Database

To connect to the database, we can use the `sqlite3` command line interface:

```
>>> sqlite3 hipparcos.db
```

```
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.
```

**Excercise 1:** Try connecting to the `hipparcos.db` database.  Use the `.help` command to determine which command to use to list the names of tables of the database and run it.

To make the output a bit more human-readable, let's run the following commands:

```
.mode column
.header on
```

In `sqlite`, we can list the columns of a table using the `PRAGMA table_info()` command:

```
sqlite> PRAGMA table_info(data);
```

**Excercise 2:** List all of the columns in the `photometry` table.  How many columns are there?

# Selecting everything from a table

```SQL
SELECT *
FROM <table_name>;
```

for example:

```SQL
SELECT *
FROM data;
```

Note that we can also limit the number of results returned:

```SQL
SELECT *
FROM data
LIMIT 10;
```

**Excercise 3:** Write a query that selects all columns from the `photometry` table and limits the results to 50 rows.

*Solution*:

```SQL
SELECT *
FROM photometry
LIMIT 50;
```

# Filtering specific columns

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>;
```

for example:

```SQL
SELECT HIP, RV, e_RV
FROM data;
```

**Exercise 4**: Write a query that selects the Hipparcos Identifier, the Hipparcos magnitude, and the variability period from the `photometry` table.  You may need to list the table's columns or refer to the documentation to check their column name.

# Filtering specific rows

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>;
```

for example:

```SQL
SELECT HIP, RV, e_RV
FROM data
WHERE RV != 0;
```

More conditions can be added to the query using the `AND` command.  There is no limit to the number of conditions you can put on a query.


```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>
AND <column_name> <condition>
...
AND <column_name> <condition>;
```

for example:
```SQL
SELECT HIP, RV, e_RV
FROM data
WHERE RV != 0
AND e_RV < 1;
```

**Exercise 5:** Write a query that selects the Hipparcos indentifier, the Johnson B apparent magnitute, the Johnson V apparent magnitude, and the variability period.  Only select sources whose variability period and B-magnitute are non-zero, and have a B-magnitude less than 10.  

# Sorting results

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>
AND <column_name> <condition>
...
AND <column_name> <condition>
ORDER BY <column_name>;
```

for example:

```SQL
SELECT HIP, Bmag, Vmag, Per
FROM photometry
WHERE Per != 0
AND Bmag != 0
AND Bmag < 10
ORDER BY Vmag;
```

**Exercise 6:** Write a query that selects the Hipparcos identifer, the Johnson B and V apparent magnitudes, and the variability period.  Only select sources whose V-magnitude is greater than 11, and whose variability type is 'P'.  Order the results by the variability period. 

# Joining Tables

<img src="https://image.slidesharecdn.com/2-background-140403152126-phpapp02/95/database-2-background-30-638.jpg?cb=1396538564">

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table1_name>
JOIN <table2_name> ON <condition>;
```

for example:

```SQL
SELECT data.HIP, pmRA, pmDE, Bmag, Vmag
FROM data
JOIN photometry ON data.HIP = photometry.HIP;
```

Note that `WHERE`, `AND`, and `ORDER BY` conditions can be applied to the query after the `JOIN`, for example:

```SQL
SELECT data.HIP, pmRA, pmDE, Bmag, Vmag
FROM data
JOIN photometry ON data.HIP = photometry.HIP
WHERE Vmag > 11
AND Bmag > 11
ORDER BY Vmag;
```

**Exercise 7:** Using `JOIN`, write a query that selects the Hipparcos identifier and the radial velocity from the `data` table, as well as the Johnson B and V apparent magnitudes from the `photometry` table. Only select sources whose radial velocity are greater than 0, and order the results by the Johnson B magnitude.

# Programming with Databases using Python

We have now learned how to retreive, filter, and sort data from a `SQL` database using the `SQLite` command line interface.  The same tasks can be performed in Python, using the `sqlite3` library.

```python
import sqlite3
```

To connect to the database, we can use the `sqlite3` `connect()` method:

```python
connection = sqlite3.connect("/Users/bourque/Desktop/swc/hipparcos.db")
```

Next, we set up a `cursor` object, which will later allow us to execute `SQL` commands:

```python
cursor = connection.cursor()
```

Executing a database query is not as simple as passing the hard-coded query to the `cursor.execute()` command:

```python
cursor.execute("<SQL COMMAND>;")
```

For example:

```python
sql_command = """
    SELECT data.HIP, RV, Bmag, Vmag
    FROM data
    JOIN photometry ON data.HIP = photometry.HIP
    WHERE RV > 0
    ORDER BY Bmag
    LIMIT 20;
    """
cursor.execute(sql_command)
```

To retreive the results of the query, we can use the `cursor.fetchall()` method.  This returns a list of tuples, with each element in the list being a row of results:

```python
results = cursor.fetchall()
for result in results:
    print(result)
```

```
(30438, 20.299999237060547, -0.4560000002384186, -0.6200000047683716)
(30438, 20.299999237060547, -0.4560000002384186, -0.6200000047683716)
(30438, 20.299999237060547, -0.4560000002384186, -0.6200000047683716)
(30438, 20.299999237060547, -0.4560000002384186, -0.6200000047683716)
(53, 11.420000076293945, 0.0, 10.960000038146973)
(53, 11.420000076293945, 0.0, 10.960000038146973)
(3937, 6.0, 0.0, 12.010000228881836)
(3937, 6.0, 0.0, 12.010000228881836)
(4725, 57.0, 0.0, 11.800000190734863)
(4725, 57.0, 0.0, 11.800000190734863)
(6115, 130.0, 0.0, 11.630000114440918)
(6115, 130.0, 0.0, 11.630000114440918)
(8939, 24.0, 0.0, 12.369999885559082)
(8939, 24.0, 0.0, 12.369999885559082)
(11517, 9.0, 0.0, 11.850000381469727)
(11517, 9.0, 0.0, 11.850000381469727)
(12886, 28.0, 0.0, 10.800000190734863)
(12886, 28.0, 0.0, 10.800000190734863)
(14574, 11.800000190734863, 0.0, 11.720000267028809)
(14574, 11.800000190734863, 0.0, 11.720000267028809)
```

With some unpacking, we can get lists of the results:

```python
HIPs = [item[0] for item in results]
RVs = [item[1] for item in results]
Bmags = [item[2] for item in results]
Vmags = [item[3] for item in results]

print(Vmags)
```

```
[-0.6200000047683716, -0.6200000047683716, -0.6200000047683716, -0.6200000047683716, 10.960000038146973, 10.960000038146973, 12.010000228881836, 12.010000228881836, 11.800000190734863, 11.800000190734863, 11.630000114440918, 11.630000114440918, 12.369999885559082, 12.369999885559082, 11.850000381469727, 11.850000381469727, 10.800000190734863, 10.800000190734863, 11.720000267028809, 11.720000267028809]
```

```python
import sqlite3

connection = sqlite3.connect("/Users/bourque/Desktop/swc/hipparcos.db")
cursor = connection.cursor()

sql_command = """
    SELECT data.HIP, RV, Bmag, Vmag
    FROM data
    JOIN photometry ON data.HIP = photometry.HIP
    WHERE RV > 0
    ORDER BY Bmag
    LIMIT 20;
    """
cursor.execute(sql_command)
results = cursor.fetchall()

HIPs = [item[0] for item in results]
RVs = [item[1] for item in results]
Bmags = [item[2] for item in results]
Vmags = [item[3] for item in results]
```

**Exercise 8:**

1. Using `sqlite3` and python, query the database for `B-V` and Absolute V mag (i.e. `VAbsMag`) measurements.
  - Select only sources that are have a Distance error (`e_Dist`) that is non-zero and less than 10. 
  - Also filter out non-zero `B-V` values
  - *Hint: You will need to join the `data` and `photometry` tables.*
  - *Hint: You will need to wrap 'B-V' in double-quotes in the `SELECT` statement to avoid errors caused by the minus-sign.*
2. Parse the results by creating a list that holds the `B-V` values and a list that holds the `VabsMag` values.
3. Plot the results using `pyplot` *(hint: It may be useful to invery the y-axis using `plt.ylim(plt.ylim()[::-1])`*

**(Bonus) Exercise 9:**

1. Execute the previous query, only this time add conditions to select only white dwarf stars (`"B-V" < 0.7`, `VAbsMag > 10`).
2. Overplot the white dwarf stars on the plot from Excercise 8 using a different marker color.

<img src="cmd_wd.png">